In [1]:
import requests
import pandas as pd

from fake_headers import Headers
from bs4 import BeautifulSoup
from nested_lookup import nested_lookup
from tqdm import tqdm

from parse_html import get_data_from_html

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 1000)

In [3]:
def nested_check(key, dct):
    """Поиск в вложенных словарях данных по ключу."""
    if dct is None:
        return None
    else:
        res = nested_lookup(key, dct)
        if res:
            return res
        else:
            return None

In [4]:
header = Headers(
        browser="chrome",  # Generate only Chrome UA
        os="win",  # Generate ony Windows platform
        headers=True  # generate misc headers
    )

In [5]:
url = 'https://spb.cian.ru/sale/flat/268076795/'

html = requests.get(url).text
soup = BeautifulSoup(html, features='html.parser')

paragraphs = [' '.join(s.findAll(text=True)) for s in soup.findAll('p')]
title = soup.html.head.title
text = soup.get_text()

In [19]:
def search_offers_desktop(target: str, page: int):
    if target == 'all':
        payload = {
        "jsonQuery": {
            "region": {
                "type": "terms",
                "value": [
                    2
                ]
            },
            "_type": "flatsale",
            "room": {
                "type": "terms",
                "value": [
                    1,
                    2,
                    3,
                    4,
                    5,
                    6
                ]
            },
            "engine_version": {
                "type": "term",
                "value": 2
            },
            "page": {
                "type": "term",
                "value": page
            }
        }
        }
    if target == 'sobstv':
        payload = {
        "jsonQuery": {
            "region": {
                "type": "terms",
                "value": [
                    2
                ]
            },
            "_type": "flatsale",
            "room": {
                "type": "terms",
                "value": [
                    1,
                    2,
                    3,
                    4,
                    5,
                    6
                ]
            },
            "engine_version": {
                "type": "term",
                "value": 2
            },
            "page": {
                "type": "term",
                "value": page
            },
            "is_by_homeowner": {
                "type": "term",
                "value": True
            }
        }
    }
    url = 'https://api.cian.ru/search-offers/v2/search-offers-desktop/'
    response = requests.post(url=url, json=payload, headers=header.generate())
    print(response)
    if response.status_code == 200:
        return response.json()
    else:
        return {}

In [20]:
r = search_offers_desktop(target='sobstv', page=1)

<Response [200]>


In [21]:
nested_check('seoData', r)[0]

{'noindex': True,
 'h1': 'Купить 1, 2, 3, 4, 5-комнатную квартиру, многокомнатную квартиру в Санкт-Петербурге',
 'similarLinks': [],
 'text': None,
 'canonical': None,
 'keywords': None,
 'description': '2 144 объявления',
 'metaDescription': 'Купить 1, 2, 3, 4, 5-комнатную квартиру, многокомнатную квартиру в Санкт-Петербурге - 2 144 объявления - база об аренде и продаже недвижимости в Санкт-Петербурге',
 'title': 'Купить 1, 2, 3, 4, 5-комнатную квартиру, многокомнатную квартиру в Санкт-Петербурге - 2 144 объявления'}

In [22]:
len(nested_check('offersSerialized', r)[0])

28

In [23]:
nested_check('offersSerialized', r)[0][7]

{'isTop3': True,
 'isAuction': True,
 'isCalltrackingEnabled': True,
 'dealRentVersion': None,
 'rosreestrCheck': {'status': 'success'},
 'livingArea': '12.0',
 'coworking': None,
 'category': 'flatSale',
 'phones': [{'countryCode': '7', 'number': '9812512365'}],
 'isColorized': False,
 'demolishedInMoscowProgramm': None,
 'land': None,
 'fromDeveloper': False,
 'shareAmount': None,
 'similar': None,
 'added': 'сегодня, 10:49',
 'decoration': None,
 'officeType': None,
 'videos': [],
 'isApartments': False,
 'showWarningMessage': False,
 'flatType': 'rooms',
 'isByCommercialOwner': None,
 'isPaid': False,
 'isInHiddenBase': False,
 'isNew': False,
 'title': 'Качественный ремонт. Гардеробная.',
 'accessType': None,
 'readyBusinessType': None,
 'addedTimestamp': 1639900187,
 'isCommercialOwnershipVerified': None,
 'workTimeInfo': None,
 'layout': None,
 'userId': 82460251,
 'promoInfo': None,
 'publishedUserId': 82460251,
 'basicProfiScore': 0,
 'newbuildingVasPromotion': None,
 'categor

In [24]:
import json
import pprint

from datetime import datetime


def get_apartment_info_from_json(json_data: dict) -> dict:
    """Получение данных о самой квартире из json-файла.
    
    @param json_data: json-файл c сырыми данными
    @return: словарь с чистыми данными
    """
     # Параметры квартиры из API
    living_area = json_data.get('livingArea', 0) or 0
    kitchen_area = json_data.get('kitchenArea', 0) or 0
    total_area = json_data.get('totalArea', 0) or 0
    floor_number = json_data.get('floorNumber', 0) or 0
    rooms_count = json_data.get('roomsCount', 0) or 0
    description = json_data.get('description', '') or ''
    bargain_terms = json_data.get('bargainTerms', {})  or {}
    if bargain_terms:
        price = bargain_terms.get('price', 0) or 0
    else:
        price = 0
    photos = json_data.get('photos', {}) or {}
    if photos:
        photos_url_list = [item.get('fullUrl', '') for item in photos]
        photos_url_list = [item for item in photos_url_list if item]
    else:
        photos_url_list = []
    # Параметры квартиры из HTML
    cian_id = json_data.get('cianId', 0) or 0
    if cian_id:
        part_of_apartment_data = get_data_from_html(cian_id)
    else:
        part_of_apartment_data = {
            'Тип жилья': '',
            'Планировка': '', 
            'Высота потолков': '', 
            'Санузел': '', 
            'Ремонт': '', 
            'Вид из окон': '', 
            'Балкон/лоджия': '', 
            'Площадь комнат': '', 
            'Отделка': '',
        }
    apartment_data = {
        'Жилая площадь, м^2': living_area,
        'Площадь кухни, м^2': kitchen_area,
        'Общая площадь, м^2': total_area,
        'Этаж': floor_number,
        'Стоимость, р.': price,
        'Количество комнат': rooms_count,
        'Описание квартиры': description,
        'Фотографии': photos_url_list,
    }
    apartment_data.update(part_of_apartment_data)
    return apartment_data
    

def get_ad_info_from_json(json_data: dict) -> dict:
    """Получение данных об объявлении из json-файла.
    
    @param json_data: json-файл c сырыми данными
    @return: словарь с чистыми данными
    """
    # Параметры объявления
    apartment_url = json_data.get('fullUrl', '')
    added_timestamp = json_data.get('addedTimestamp', '')
    user = json_data.get('user', {})
    if user:
        user_type = user.get('userType', '')
    else:
        user_type = ''
    ad_data = {
        'Ссылка на объявление': apartment_url,
        'Дата публикации объявления': datetime.fromtimestamp(
            added_timestamp).strftime("%Y-%m-%d %H:%M:%S"),
        'Тип автора объявления': user_type,
    }
    return ad_data
    

def get_building_info_from_json(json_data: dict) -> dict:
    """Получение данных о доме из json-файла.
    
    @param json_data: json-файл c сырыми данными
    @return: словарь с чистыми данными
    """
    # Параметры дома
    building = json_data.get('building', {}) or {}
    if building:
        passenger_lifts_count = building.get('passengerLiftsCount', 0) or 0
        build_year = building.get('buildYear', '') or ''
        cargo_lifts_count = building.get('cargoLiftsCount', 0) or 0
        floors_count = building.get('floorsCount', 0) or 0
        material_type = building.get('materialType', '')  or ''
    else:
        passenger_lifts_count = 0
        build_year = ''
        cargo_lifts_count = 0
        floors_count = 0
        material_type = ''
    building_data = {
        'Количество пассажирских лифтов': passenger_lifts_count,
        'Год постройки': build_year,
        'Количество грузовых лифтов': cargo_lifts_count,
        'Количество этажей': floors_count,
        'Технология строительства': material_type,
    }
    return building_data
    

def get_location_info_from_json(json_data: dict) -> dict:
    """Получение данных о местоположении квартиры из json-файла.
    
    @param json_data: json-файл c сырыми данными
    @return: словарь с чистыми данными
    """
     # Параметры расположения
    geo = json_data.get('geo', {}) or {}
    if geo:
        user_input = geo.get('userInput', '')
        coordinates = geo.get('coordinates', {}) or {}
        address = geo.get('address', []) or []
        if coordinates:
            lng = coordinates.get('lng', 0) or 0
            lat = coordinates.get('lat', 0) or 0
        else:
            lng = 0
            lat = 0
        if address:
            location_flag = False
            okrug_flag = False
            raion_flag = False
            street_flag = False
            house_flag = False
            metro_station_flag = False
            for item in address:
                if 'type' in item:
                    if item['type'] == 'location':
                        location = item['name']
                        location_flag = True
                    if item['type'] == 'okrug':
                        okrug = item['name']
                        okrug_flag = True
                    if item['type'] == 'raion':
                        raion = item['name']
                        raion_flag = True
                    if item['type'] == 'street':
                        street = item['name']
                        street_flag = True
                    if item['type'] == 'house':
                        house = item['name']
                        house_flag = True
                    if item['type'] == 'metro':
                        metro_station = item['name']
                        metro_station_flag = True
                else:
                    location = ''
                    district = ''
                    raion = ''
                    street = ''
                    house = ''
                    metro_station = ''
            if not location_flag:
                location = ''
            if not okrug_flag:
                okrug = ''
            if not raion_flag:
                raion = ''
            if not street_flag:
                street = ''
            if not house_flag:
                house = ''
            if not metro_station_flag:
                metro_station = ''
        else:
            location = ''
            district = ''
            raion = ''
            street = ''
            house = ''
            metro_station = ''
    else:
        user_input = ''
        lng = 0
        lat = 0
        location = ''
        okrug = ''
        raion = ''
        street = ''
        house = ''
        metro_station = ''
    transport_type = {'walk': 'пешком', 'transport': 'на транспорте'}
    if geo:
        undergrounds = geo.get('undergrounds', []) or []
        if undergrounds:
            undergrounds_proximity = []
            for item in undergrounds:
                if 'name' in item and 'transportType' in item and 'time' in item:
                    line = f"{item['name']}, {item['time']} минут(ы) {transport_type[item['transportType']]}"
                    undergrounds_proximity.append(line)
        else:
            undergrounds_proximity = []
    else:
        undergrounds_proximity = []
    location_data = {
        'Широта': lat,
        'Долгота': lng,
        'Адрес, введенный пользователем': user_input,
        'Город': location,
        'Округ': okrug,
        'Район': raion,
        'Улица': street,
        'Дом': house,
        'Станция метро': metro_station,
        'Близость к метро': undergrounds_proximity,
    }
    return location_data
    
    
def get_all_apartment_info_from_json(json_data: dict) -> dict:
    """Получение полных данных о квартире из json-файла.
    
    @param json_data: json-файл c сырыми данными
    @return: словарь с чистыми данными
    """
    # Полный набор данных
#     all_data = {
#         'Характеристики квартиры': get_apartment_info_from_json(json_data),
#         'Характеристики дома': get_building_info_from_json(json_data),
#         'Характеристики расположения': get_location_info_from_json(json_data),
#         'Характеристики объявления': get_ad_info_from_json(json_data),
#     }
    all_data = get_apartment_info_from_json(json_data)
    all_data.update(get_building_info_from_json(json_data))
    all_data.update(get_location_info_from_json(json_data))
    all_data.update(get_ad_info_from_json(json_data))
    return all_data

In [25]:
t = nested_check('offersSerialized', r)[0][7]


pprint.pprint(get_all_apartment_info_from_json(t), indent=4)

{   'Адрес, введенный пользователем': 'Россия, Санкт-Петербург, '
                                      'Орлово-Денисовский проспект, 19к3',
    'Балкон/лоджия': '',
    'Близость к метро': [],
    'Вид из окон': 'Во двор',
    'Высота потолков': '',
    'Год постройки': '',
    'Город': 'Санкт-Петербург',
    'Дата публикации объявления': '2021-12-19 10:49:47',
    'Долгота': 30.26956,
    'Дом': '19к3',
    'Жилая площадь, м^2': '12.0',
    'Количество грузовых лифтов': 2,
    'Количество комнат': 1,
    'Количество пассажирских лифтов': 3,
    'Количество этажей': 25,
    'Общая площадь, м^2': '37.4',
    'Округ': 'Коломяги',
    'Описание квартиры': 'Агентов прошу не звонить, в их помощи не нуждаюсь. \n'
                         'Есть свой агент.\n'
                         '\n'
                         'В квартире никто не прописан. Свободная продажа. Без '
                         'обременений. Документы на руках.\n'
                         '\n'
                         'Застрой

In [14]:
page = nested_check('offersSerialized', r)[0]

len(page)

28

In [26]:
pages = []

for page in tqdm(range(1, 10+1)):
    one_page = search_offers_desktop(target='sobstv', page=page) 
    pages.append(nested_check('offersSerialized', one_page)[0])

 10%|████████▎                                                                          | 1/10 [00:01<00:16,  1.82s/it]

<Response [200]>


 20%|████████████████▌                                                                  | 2/10 [00:03<00:15,  1.89s/it]

<Response [200]>


 30%|████████████████████████▉                                                          | 3/10 [00:04<00:11,  1.61s/it]

...

<Response [200]>


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:12<00:01,  1.26s/it]

<Response [200]>


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:13<00:00,  1.39s/it]

<Response [200]>


In [27]:
pages = [item for sublist in pages for item in sublist]

In [28]:
len(pages)

280

In [29]:
ads = []

for ad in tqdm(pages):
    ads.append(get_all_apartment_info_from_json(ad))

100%|████████████████████████████████████████████████████████████████████████████████| 280/280 [01:21<00:00,  3.43it/s]


In [30]:
df = pd.json_normalize(ads)

In [31]:
df

,"Жилая площадь, м^2","Площадь кухни, м^2","Общая площадь, м^2",Этаж,"Стоимость, р.",Количество комнат,Описание квартиры,Фотографии,Тип жилья,Планировка,Высота потолков,Санузел,Ремонт,Вид из окон,Балкон/лоджия,Площадь комнат,Отделка,Количество пассажирских лифтов,Год постройки,Количество грузовых лифтов,Количество этажей,Технология строительства,Широта,Долгота,"Адрес, введенный пользователем",Город,Округ,Район,Улица,Дом,Станция метро,Близость к метро,Ссылка на объявление,Дата публикации объявления,Тип автора объявления
0,70.0,30.0,117.2,2,28800000,3,"Интересная квартира в доме бизнес-класса ЖК ""Преображенский""\n\nХарактеристика локации:\n- самый центр Васильевского острова\n- до развязки и Невского пр. 10 минут транспортом\n- равная доступность всех районов города\n- близость садиков, школ, ВУЗов\n- близость музеев и общественных пространств\n- место прогулок: скверы, сады, набережные\n- в 3-5 минутах ходьбы сетевые магазины\n\nХарактеристика комплекса:\n- камерный дом всего на 124 квартиры\n- закрытая охраняемая территория\n- благоустроенные дворы\n- открытая парковка на территории ЖК\n- велопарковка\n- спортивные и детские площадки\n- видеонаблюдение: периметр, двор, подъезд, лифт\n\nХарактеристика дома:\n- монолитный новый дом 2016 года постройки\n- возможность установки кондиционера\n- консьерж и охрана\n- паркинг \n- достойные соседи и окружение \n- заселён на 98%\n\nХарактеристика квартиры:\n- вариативная планировка\n- окна на восточную сторону\n- 2 санузла\n- места для организации хранения\n- можно перенести и/или увелич...","[https://cdn-p.cian.site/images/99/308/911/kvartira-sanktpeterburg-bolshoy-vasilevskogo-ostrova-prospekt-1198039911-1.jpg, https://cdn-p.cian.site/images/21/408/911/kvartira-sanktpeterburg-bolshoy-vasilevskogo-ostrova-prospekt-1198041259-1.jpg, https://cdn-p.cian.site/images/91/408/911/kvartira-sanktpeterburg-bolshoy-vasilevskogo-ostrova-prospekt-1198041945-1.jpg, https://cdn-p.cian.site/images/83/408/911/kvartira-sanktpeterburg-bolshoy-vasilevskogo-ostrova-prospekt-1198043870-1.jpg, https://cdn-p.cian.site/images/65/408/911/kvartira-sanktpeterburg-bolshoy-vasilevskogo-ostrova-prospekt-1198045622-1.jpg, https://cdn-p.cian.site/images/70/508/911/kvartira-sanktpeterburg-bolshoy-vasilevskogo-ostrova-prospekt-1198050726-1.jpg, https://cdn-p.cian.site/images/59/408/911/kvartira-sanktpeterburg-bolshoy-vasilevskogo-ostrova-prospekt-1198049574-1.jpg, https://cdn-p.cian.site/images/89/408/911/kvartira-sanktpeterburg-bolshoy-vasilevskogo-ostrova-prospekt-1198049872-1.jpg, https://cdn-p.cian....",Вторичка,Смежно-изолированная,"2,77 м",1 раздельный,Без ремонта,На улицу,,,,1,2016,1,9,monolith,59.929529,30.254001,"Россия, Санкт-Петербург, Большой проспект Васильевского острова, 84",Санкт-Петербург,№ 7,Василеостровский,Большой Васильевского острова,84,Василеостровская,"[Василеостровская, 7 минут(ы) на транспорте, Приморская, 7 минут(ы) на транспорте]",https://spb.cian.ru/sale/flat/268076795/,2021-12-19 01:45:19,realtor_not_commerce
1,75.0,15.0,132.1,3,39600000,3,"Собственник,предлагает к продаже 3-х комнатная двухсторонная квартира, 132,1м2, расположена на 3 этаже в центре Петроградского района, угол Малого пр-та и ул.Подковырова 16/65. Один собственник. Современный благоустроенный дом, дверь в квартиру двойная металлическая,лифт есть прямо из помещения паркинга, в доме 1 парадная с улицы. Планировка, комнаты изолированные. Ремонт выполнен из качественных материалов, плитка Испанская, радиаторы отопления - Германия, полы паркетная доска, дубовые межкомнатные двери. Большой балкон в тихий двор. Выход на балкон из гостиной. Окна кухни гостиной и одной спальни выходят во двор южная сторона,в квартире солнечно и даже зимой много дневного света. Оборудованная кухня, фасад натуральное дерево беленый дуб, посудомойка, индукционная плита. В квартире 3 санузла (2 санузла расположены в спальнях) в одном душевая кабина. Есть гардеробная, в шаговой доступности шкала, детские сады, магазины продовольственные и хозяйственные

In [32]:
df.shape

(280, 35)